Michael Ruggiero <br>
michael@mcruggiero.com <br>
2021.05.07 <br>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Summary-of-Problem" data-toc-modified-id="Summary-of-Problem-1">Summary of Problem</a></span></li><li><span><a href="#Libraries-Used" data-toc-modified-id="Libraries-Used-2">Libraries Used</a></span></li><li><span><a href="#Data-Generator" data-toc-modified-id="Data-Generator-3">Data Generator</a></span></li><li><span><a href="#Generate-CSVs-with-shipping-rules" data-toc-modified-id="Generate-CSVs-with-shipping-rules-4">Generate CSVs with shipping rules</a></span></li><li><span><a href="#Genetic-Algorithm" data-toc-modified-id="Genetic-Algorithm-5">Genetic Algorithm</a></span></li></ul></div>

## Summary of Problem

This problem was sketched out for me by Raghu. The following code is not the ideal solution. Solutions to the 3d bin packing problem exist on github, but I thought using a genetic algorithm would allow me to showcase more data-science applications.

## Libraries Used

In [23]:
from string import ascii_letters

import numpy as np
from numpy.random import randint, choice

import pandas as pd
import csv 

from matplotlib import pyplot as plt

## Data Generator

These are our classes to create our problem space. CSVs can be imported to test
particular problems

In [130]:
class OrderGenerator:
    """
    The purchase order generator class, creates a .csv that can be imported 
    into box solver
    """
    def __init__(self, 
                 max_items: int = 50,
                 min_items: int = 10,
                 max_dim: int = 60,
                 min_dim: int = 5,
                 max_weight: int = 1500,
                 min_weight: int = 50) -> None:
        """
        Order generator constructor
        :param max_items:   integer, max number of items in purchase order
        :param min_items:   integer, min number of items in purchase order
        :param max_dim:     integer, max number of centimeter dimension
        :param min_dim:     integer, min number of centimeter dimension
        :param max_weight:  integer, max number of grams for each object
        :param min_weight:  integer, min number of grams for each object
        """
        # First set number of items
        self.items = randint(min_items, max_items)
        self.max_dim = max_dim
        self.min_dim = min_dim
        self.max_weight = max_weight
        self.min_weight = min_weight
        
        # Next create a dictionary with random name and dimensions
        self.purchase_order = {}
        for i in range(self.items):
            self.purchase_order[i] = entry = {}
            entry["name"] = "".join(choice(list(ascii_letters), size = 12))
            entry["x_dim"] = randint(self.min_dim, self.max_dim)
            entry["y_dim"] = randint(self.min_dim, self.max_dim)
            entry["z_dim"] = randint(self.min_dim, self.max_dim)
            entry["weight"] = randint(self.min_weight, self.max_weight)
            
    def make_csv(self) -> None:
        """
        Creates csv from PO.
        """
        # Going through pandas eats a little bit of time, but more readable
        self.df = pd.DataFrame.from_dict(self.purchase_order).T
        self.df.to_csv("purchase_order.csv")
        
class ShippingContainers:
    """
    The shipping container rule class, creates .csv that has all of the packing
    rules
    """
    def __init__(self,
                 max_types: int = 10,
                 min_types: int = 4,
                 max_dim: int = 300,
                 min_dim: int = 50,
                 max_weight: int = 100_000,
                 weight_range: float = .75,
                 max_cost: int = 50,
                 min_cost: int = 5) -> None:
        """
        Shipping rules generator
        :param max_types:    integer, max number of box classes
        :param min_types:    integer, min number of box classes
        :param max_dim:      integer, max number of centimeter dimension
        :param min_dim:      integer, min number of centimeter dimension
        :param max_weight:   integer, max weight for boxes in grams
        :param weight_range: float, range to make for smaller boxes
        :param max_cost:     interger, max cost to ship box
        """
        self.types = randint(min_types, max_types)
        self.min_dim = min_dim
        self.costs = np.linspace(min_cost, max_cost, self.types, dtype = int)
        
        # We will apply linspace scaling between types.
        self.max_x_dim = randint(max_dim/2, max_dim)
        self.max_y_dim = randint(max_dim/2, max_dim)
        self.max_z_dim = randint(max_dim/2, max_dim)
        
        # Dimensions list this syntax is not perfect, repay debt later
        self.x_dims = np.linspace(min_dim, self.max_x_dim, self.types, dtype = int)
        self.y_dims = np.linspace(min_dim, self.max_y_dim, self.types, dtype = int)
        self.z_dims = np.linspace(min_dim, self.max_z_dim, self.types, dtype = int)
        
        # Not a perfect solution for weight range, but will do for randomization
        self.weights = np.linspace(max_weight * weight_range, 
                                   max_weight, 
                                   self.types,
                                   dtype = int)

        # Next create dictionary with scaling
        self.shipping_types = {}
        for i in range(self.types):
            self.shipping_types[i] = entry = {}
            entry["type"]  = i
            entry["x_dim"] = self.x_dims[i]
            entry["y_dim"] = self.y_dims[i]
            entry["z_dim"] = self.z_dims[i]
            entry["volume"] = self.x_dims[i] * self.y_dims[i] * self.z_dims[i]
            entry["max_weight"] = self.weights[i]
            entry["cost"] = self.costs[i]
            
    def make_csv(self) -> None:
        """
        Creates csv from shipping types.
        """
        self.df = pd.DataFrame.from_dict(self.shipping_types).T
        self.df.to_csv("shipping_types.csv")

## Generate CSVs with shipping rules

In [131]:
OrderGenerator().make_csv()
ShippingContainers().make_csv()

In [132]:
shipping = pd.read_csv("shipping_types.csv", index_col = 0)
shipping.head()

,type,x_dim,y_dim,z_dim,volume,max_weight,cost
0,0,50,50,50,125000,75000,5
1,1,92,89,100,818800,81250,16
2,2,134,128,150,2572800,87500,27
3,3,176,167,200,5878400,93750,38
4,4,219,206,251,11323614,100000,50


In [126]:
purchase = pd.read_csv("purchase_order.csv", index_col = 0)
purchase.head()

,name,x_dim,y_dim,z_dim,weight
0,rNOCpTiLgtAX,7,47,49,218
1,tKEtSTVEGfNU,38,6,29,649
2,gChIxeGSorQP,22,39,52,164
3,lwJDjIpiMXAt,10,6,21,115
4,ScYZMfCepbVk,44,29,45,685


## Genetic Algorithm

Under the current definition of our problem, the exact solution will be non-linear NP, scaling with the number of items in the purchase order. To find a reasonable solution we will use a genetic algorithm design. There are other options for our algorithm, but I think this design will allow me to experiment with various machine learning techniques to find the best solution.

In [ ]:
class GeneticPacking:
    """
    A genetic packing algorithm to solve the 3D bin packing problem
    """
    def __init__(self) -> None:
        """
        """
        pass
    
    def random_generator(self) -> None:
        """
        """
        pass
    
    def fitness(self) -> int:
        """
        """
        pass
    
    def select(self) -> None:
        """
        """
        pass
    
    def mate(self) -> None:
        """
        """
        pass
    
    def mutation(self) -> None:
        pass
    